In [89]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

from IPython.display import display
from ipywidgets import FloatProgress
import json
from multiprocessing import Pool
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
import sklearn
import sys
sys.path.insert(0, '../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# RGES Permutation Testing

This notebook implements a multithreaded permutation test for RGES Scores. The method is based on Sirota et al's<sup>1</sup> computation approach to drug validation with gene expression data. The input files are:

**Phenotype Signature**

A tab-separated format with the following columns:
- ```entrezgene```: Entrez gene ID numbers
- ```log2FoldChange```: Fold change for the upregulated genes.
- ```log2fc.y```: The **unsigned** fold change for the downregulated genes.

**Drug Profiles**

This file should be in [GCTX](https://clue.io/connectopedia/gctx_format) format. It should be the immediate output of ranking another ```GCTX``` with the [cmapR](https://github.com/cmap/cmapR) ```rank.gct``` method, which takes a ```GCTX``` file with differential expression data as input and returns a matrix of the same size with the columns containing ranks instead.

**True Scores**

A ```.json``` file containing the true scores calculated previously for the **phenotype signature** and **drug profiles**. The keys are drug profile names and the values are RGES scores.

## Preprocessing

### Load The Pipeline

In [6]:
from RGES.DiffEx import DiffEx
from RGES.L1KGCT import L1KGCTX
from RGES.Score import score

### Data Filepaths

In [73]:
PHENOTYPE_SIGNATURE_PATH = "/home/jovyan/oncogxA/Alex/l1k/DEG_SC_5um_entrezgene.txt"
DRUG_PROFILE_PATH = "/home/jovyan/oncogxA/Alex/l1k/10x_ilincs_sigs_top500_ranked_n500x978.gctx"
SCORES_PATH = "LINCS_top500_scores.json"
#SCORES_PATH = "/home/jovyan/oncogxA/Alex/l1k/LINCS_FULL_GEO_RANKED/LINCS_landmarks_python_scores.json"

OUTPATH = "ilincs_top500_permutations.json"

### Loading/Processing Input Data

In [74]:
DE = DiffEx(PHENOTYPE_SIGNATURE_PATH)
LINCS = L1KGCTX(DRUG_PROFILE_PATH)
TRUE_SCORES = json.loads(open(SCORES_PATH).read())

merge_l2fc = lambda x: -1.0*x['log2fc.y'] if not np.isnan(x['log2fc.y']) else x['log2FoldChange']
DE.data['log2FoldChange'] = DE.data.apply(merge_l2fc, axis=1)

## Permutation Testing Implementation

### Multithreaded Scoring

In [26]:
def mt_score_CHILD(signame):
    """Returns the RGES score for signame based on DE and LINCS"""
    return ((signame, score(DE, LINCS, signame)))

def mt_score(procs):
    """Returns a dictionary of {drug_profile: score}"""
    p = Pool(processes=procs)
    res = p.map(mt_score_CHILD, list(LINCS.data))
    return {r[0]: r[1] for r in res}

### Permuting The Disease Signatures

Inspired by https://stackoverflow.com/questions/15772009/shuffling-permutating-a-dataframe-in-pandas

In [62]:
def shuffle_sigs():
    LINCS.data = LINCS.data.apply(sklearn.utils.shuffle, axis=0)
    LINCS.data.index = map(str, sorted(map(int, list(LINCS.data.index))))

### Permutation Test Driver

In [72]:
PERMUTATIONS = 100
PROCESSES = 1

prog_bar = FloatProgress(min=0, max=PERMUTATIONS)
display(prog_bar)

res_d = {signame: [] for signame in list(LINCS.data)}

for i in range(PERMUTATIONS):
    shuffle_sigs()
    p_res = mt_score(PROCESSES)
    for signame in p_res.keys():
        res_d[signame].append(p_res[signame])
    prog_bar.value += 1
        
open(OUTPATH, 'w').write(json.dumps(res_d))

1080215

## Permutation Test Analysis

### Load Permutation Data and True Scores

In [75]:
TRUE_SCORES = json.loads(open(SCORES_PATH).read())
PERM_SCORES = json.loads(open(OUTPATH).read())

### Calculate P-Values (unadjusted) For Each Drug Profile

In [93]:
pvals = {}  #{drug_profile: pval}

for signame in TRUE_SCORES.keys():
    score = TRUE_SCORES[signame]
    if score < 0:
        continue  #Don't care about negative RGES
    perms = PERM_SCORES[signame]
    extr_count = float(sum([1 if p >= score else 0 for p in perms]))
    pvals[signame] = extr_count/len(perms)
    
pvals = pd.DataFrame(list(pvals.items()), columns=["drug_profile", "pval"])
stats = importr('stats')
pvals['bh'] = stats.p_adjust(FloatVector(pvals['pval']), method='BH')
pvals[pvals["bh"]<0.05]

,drug_profile,pval,bh
6,LINCSCP_20399,0.0,0.0
13,LINCSCP_15728,0.0,0.0
14,LINCSCP_55167,0.0,0.0
26,LINCSCP_23402,0.0,0.0
32,LINCSCP_9809,0.0,0.0
37,LINCSCP_55145,0.0,0.0
43,LINCSCP_52113,0.0,0.0
49,LINCSCP_9627,0.0,0.0


1. Sirota, M., Dudley, J.T., Kim, J., Chiang, A.P., Morgan, A.A., Sweet-Cordero, A., Sage, J., and Butte, A.J. (2011). Discovery and Preclinical Validation of Drug Indications Using Compendia of Public Gene Expression Data. Science Translational Medicine 3, 96ra77-96ra77.